In [45]:
import sys
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
sys.path.append("../scripts")
from Eda import Eda as eda
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.cluster import KMeans

In [2]:
data = pd.read_csv("../data/Week1_challenge_data_source.csv")
eda = eda(data)
data = eda.convertbyte_scale(data, '(Bytes)', '(MB)', 1000000)

In [3]:
df = pd.DataFrame()

In [8]:
df = pd.DataFrame()
datas = pd.DataFrame()
df['Avg RTT'] = data['Avg RTT DL (ms)'] + data['Avg RTT UL (ms)']
df['Avg TCP'] = data['TCP DL Retrans. Vol (MB)'] + data['TCP UL Retrans. Vol (MB)']
df['Avg TP'] = data['Avg Bearer TP DL (kbps)'] + data['Avg Bearer TP UL (kbps)']
df['Avg TP'] = df['Avg TP']/1000
data['Session frequency'] = data.groupby(['Bearer Id'])['Dur. (ms)'].transform('count')
datas['Session frequency'] = data['Session frequency']
data['Total_Data_Volume(MB)'] = data['Total DL (MB)'] + data['Total UL (MB)']
datas["Total_Data"] = data['Total_Data_Volume(MB)']
datas['Dur. (s)'] = data['Dur. (ms)']/1000
datas['MSISDN/Number'] = data['MSISDN/Number']
datas['Handset_type'] = data['Handset Type']
datas['MSISDN/Number'] = data['MSISDN/Number']


num_col, cat_col = eda.col_assign(df)
df = eda.fix_num_cols(df, num_col)
outliers = ['Avg RTT','Avg TCP','Avg TP']
df = eda.fix_outlier(df, outliers)
df.dropna(inplace=True)
datas.dropna(inplace=True)

In [9]:
km = KMeans(n_clusters=3,)
prediction = km.fit_predict(df[['Avg TP', 'Avg TCP', 'Avg RTT']])
df['cluster'] = prediction 

df1 = df[df.cluster==0]
df2 = df[df.cluster==1]
df3 = df[df.cluster==2]
print('For Cluster 0')

print(df1.describe())
print('For Cluster 1')

print(df2.describe())
print('For Cluster 2')

print(df3.describe())

For Cluster 0
            Avg RTT       Avg TCP        Avg TP  cluster
count  86949.000000  86949.000000  86949.000000  86949.0
mean      45.866542     19.816147      9.425343      0.0
std       16.541905     10.472051     16.636066      0.0
min        0.000000      0.000000      0.001000      0.0
25%       32.000000     25.716954      0.090000      0.0
50%       43.000000     25.716954      0.116000      0.0
75%       58.000000     25.716954     15.070475      0.0
max       84.000000     25.716954     71.093000      0.0
For Cluster 1
            Avg RTT       Avg TCP        Avg TP  cluster
count  56283.000000  56283.000000  56283.000000  56283.0
mean     121.746836     18.104664     11.760849      1.0
std       15.091781     10.810342     18.169687      0.0
min       82.000000      0.000000      0.000000      1.0
25%      122.000000      4.592500      0.087000      1.0
50%      127.024885     25.716954      0.126000      1.0
75%      127.024885     25.716954     18.495000      1.0
max

In [10]:
df['cluster'].value_counts()

0    86949
1    56283
2     6769
Name: cluster, dtype: int64

In [11]:
predictions = km.fit_predict(datas[['Total_Data', 'Dur. (s)', 'Session frequency']])
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [15]:

datas['cluster'] = predictions

dfs = datas[datas.cluster==0]
dfs1 = datas[datas.cluster==1]
dfs2 = datas[datas.cluster==2]
print('For Cluster 0')
print(dfs.describe())

print('For Cluster 1')
print(dfs1.describe())

print('For Cluster 2')
print(dfs2.describe())

For Cluster 0
       Session frequency     Total_Data       Dur. (s)  MSISDN/Number  \
count      112176.000000  112176.000000  112176.000000   1.121760e+05   
mean        37528.746327     495.635803     115.815487   3.667854e+10   
std          1968.110040     244.402764      84.987232   1.006089e+12   
min         35114.000000      28.956000       7.142000   3.360100e+10   
25%         35114.000000     284.784250      75.496500   3.365097e+10   
50%         37251.000000     496.677000      92.701000   3.366366e+10   
75%         39915.000000     706.413000     147.673500   3.368369e+10   
max         39915.000000     955.985000    1859.336000   3.370000e+14   

        cluster  
count  112176.0  
mean        0.0  
std         0.0  
min         0.0  
25%         0.0  
50%         0.0  
75%         0.0  
max         0.0  
For Cluster 1
       Session frequency   Total_Data     Dur. (s)  MSISDN/Number  cluster
count        7050.000000  7050.000000  7050.000000   7.050000e+03   7050.0
me

In [25]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148506 entries, 0 to 149999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Session frequency  148506 non-null  float64
 1   Total_Data         148506 non-null  float64
 2   Dur. (s)           148506 non-null  float64
 3   MSISDN/Number      148506 non-null  float64
 4   Handset_type       148506 non-null  object 
 5   cluster            148506 non-null  int32  
 6   Engagement_score   7050 non-null    float64
dtypes: float64(5), int32(1), object(1)
memory usage: 12.5+ MB


In [14]:
import numpy as np

In [18]:
df3['Experience_score'] =np.sqrt(((df3['Avg RTT'] +df3['Avg TCP'] + df3['Avg TP']).pow(2)) - (df3['cluster'].pow(2)))

C:\Users\Victor Olufemi\AppData\Local\Temp\ipykernel_16124\2317214630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Experience_score'] =np.sqrt(((df3['Avg RTT'] +df3['Avg TCP'] + df3['Avg TP']).pow(2)) - (df3['cluster'].pow(2)))


In [26]:
dfs1['Engagement_score'] =np.sqrt(((dfs1['Total_Data'] +dfs1['Dur. (s)'] + dfs1['Session frequency']).pow(2)) - (dfs1['cluster'].pow(2)))

C:\Users\Victor Olufemi\AppData\Local\Temp\ipykernel_16124\1008134268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs1['Engagement_score'] =np.sqrt(((dfs1['Total_Data'] +dfs1['Dur. (s)'] + dfs1['Session frequency']).pow(2)) - (dfs1['cluster'].pow(2)))


In [46]:
df_data = pd.concat([df3, dfs1], axis=1,join='inner')
def minmax_scale(datas, col_names):
    scaler = MinMaxScaler(feature_range=(0, 1))
    datas[col_names] = scaler.fit_transform(datas[col_names])
minmax_scale(df_data, ['Engagement_score', 'Experience_score'])

In [47]:
df_data['Satisfactory_score']= (df_data['Experience_score']+ df_data['Engagement_score'])/2

In [48]:
df_data.drop('cluster',axis=1, inplace=True)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1698 entries, 151 to 149835
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Avg RTT             1698 non-null   float64
 1   Avg TCP             1698 non-null   float64
 2   Avg TP              1698 non-null   float64
 3   Experience_score    1698 non-null   float64
 4   Session frequency   1698 non-null   float64
 5   Total_Data          1698 non-null   float64
 6   Dur. (s)            1698 non-null   float64
 7   MSISDN/Number       1698 non-null   float64
 8   Handset_type        1698 non-null   object 
 9   Engagement_score    1698 non-null   float64
 10  Satisfactory_score  1698 non-null   float64
dtypes: float64(10), object(1)
memory usage: 159.2+ KB


In [64]:
df_data.columns

Index(['Avg RTT', 'Avg TCP', 'Avg TP', 'Experience_score', 'Session frequency',
       'Total_Data', 'Dur. (s)', 'MSISDN/Number', 'Handset_type',
       'Engagement_score', 'Satisfactory_score', 'cluster'],
      dtype='object')

Modelling

In [49]:
from sklearn.model_selection import train_test_split
X = df_data.drop(['Satisfactory_score','Handset_type', 'MSISDN/Number'], axis=1)
y = df_data.Satisfactory_score
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1358, 8) (340, 8) (1358,) (340,)


In [50]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)

In [51]:
from sklearn.metrics import r2_score
pred = lr.predict(X_val)
r2 = r2_score(y_val, pred)
print(r2)

1.0


In [52]:
km = KMeans(n_clusters=2)
preds = km.fit_predict(df_data[['Engagement_score', 'Experience_score']])
df_data['cluster']  = preds

In [53]:
satisfied_per_cluster = df_data.groupby('cluster').agg({'Satisfactory_score':["median","mean"]})
experienced_per_cluster = df_data.groupby('cluster').agg({'Experience_score':["median","mean"]})
print(satisfied_per_cluster)
print(experienced_per_cluster)

        Satisfactory_score          
                    median      mean
cluster                             
0                 0.670999  0.678418
1                 0.286742  0.291556
        Experience_score          
                  median      mean
cluster                           
0               0.413983  0.424279
1               0.406668  0.404696


In [55]:
final_table = pd.DataFrame()
final_table['user_id'] = df_data['MSISDN/Number']

In [58]:
cols = ['Experience_score','Engagement_score','Satisfactory_score']

In [59]:
for i in cols:
    final_table[i] = df_data[i]

In [60]:
final_table

,user_id,Experience_score,Engagement_score,Satisfactory_score
151,3.365039e+10,0.422655,0.265614,0.344134
435,3.366411e+10,0.376428,0.933443,0.654936
468,3.365856e+10,0.684587,0.871892,0.778239
472,3.366735e+10,0.539031,0.875386,0.707209
476,3.366441e+10,0.516551,0.963717,0.740134
...,...,...,...,...
149675,3.369974e+10,0.569508,0.934429,0.751969
149676,3.366393e+10,0.632806,0.161989,0.397397
149798,3.362215e+10,0.532814,0.897508,0.715161
149818,3.365976e+10,0.444268,0.942144,0.693206


In [61]:
final_table.to_csv('final_table.csv')

In [62]:
import joblib

joblib.dump(lr, "clf.pkl")

['clf.pkl']